In [14]:
import googlemaps
import requests as re
from bs4 import BeautifulSoup
import json
import os
import folium
from selenium import webdriver
from datetime import datetime
from math import cos ,sqrt
import numpy as np
import openpyxl

In [15]:
os.chdir(r'..\data1')
f = open('geo_seoul.json', encoding='utf8')
geo_seoul = json.load(f)
f.close()
geo_seoul = geo_seoul['features'][2]

In [16]:

arr=[]
arr1 = np.linspace(37.5334,37.45903,10)
arr2 = np.linspace(127.110009,127.01696,10)
dif1 = abs(arr1[0]-arr1[1])/2
dif2 = abs(arr2[0]-arr2[1])/2
for a1 in arr1:
    for a2 in arr2:
        arr.append([a1,a2])
print(127.12164-2*dif2)

127.11130122222222


In [17]:
index = []
def isNum(a):
    try:
        float(a)
        return True
    except:
        return False
cate= ["편의점","스타벅스","지하철","패스트푸드"]
while len(index)!=4:
    a = input("0~5사이의 중요도를 입력하세요!! "+cate[len(index)])
    if isNum(a):
        a=int(a)
    else:
        print("다시입력하세요")
        continue
    if 0<=a<6:
        index.append(a)
        continue
    print("다시입력하세요")

0~5사이의 중요도를 입력하세요!! 편의점1
0~5사이의 중요도를 입력하세요!! 스타벅스1
0~5사이의 중요도를 입력하세요!! 지하철1
0~5사이의 중요도를 입력하세요!! 패스트푸드1


In [18]:
def conviScore(u,dif1,dif2,i):
    os.chdir(r"..\data1")                                   
    wb = openpyxl.load_workbook('convi.xlsx')                            # 편의점 좌표가 있는 파일 불러오기
    sh = wb['Sheet']
    score = 0                                               
    for li in range(1,677):                                              # 편의점 좌표 하나하나 반복문으로 출력
        loc = [sh.cell(li,1).value ,sh.cell(li,2).value]
        if (a[1]-dif2<loc[1]<a[1]+dif2) &(a[0]-dif1<loc[0]<a[0]+dif1):  # 편의점 좌표가 사각 격자 범위에 있는지 확인
            score+=(0.5-get_dis(u,loc))*i                                # (최대거리 - 편의점과의 거리) X 중요도 값
#         folium.Circle(loc,color="green", popup='편의점', tooltip="",radius=10).add_to(my_map)
    return score                                                         # 합계 출력
def starBScore(u,dif1,dif2,i):                                           # 위 함수와 같은 함수(스타벅스)
    os.chdir(r"..\data1")
    wb = openpyxl.load_workbook('starbucks.xlsx')
    sh = wb['starbucks']
    score = 0
    for li in range(2,83):
        loc = [float(sh.cell(li,3).value) ,float(sh.cell(li,4).value)]
        if (a[1]-dif2<loc[1]<a[1]+dif2) &(a[0]-dif1<loc[0]<a[0]+dif1):
            score+=(0.5-get_dis(u,loc))*i
#         folium.Circle(loc,color="yellow", popup='스타벅스', tooltip="",radius=10).add_to(my_map)
    return score
def subwayScore(u,dif1,dif2,i):    # 위 함수와 같은 함수(지하철역)
    os.chdir(r"..\data1")
    wb = openpyxl.load_workbook('subway.xlsx')
    sh = wb['subway']
    score = 0
    for li in range(1,14):
        loc = [float(sh.cell(li,1).value) ,float(sh.cell(li,2).value)]
        if (a[1]-dif2<loc[1]<a[1]+dif2) &(a[0]-dif1<loc[0]<a[0]+dif1):
            score+=(0.5-get_dis(u,loc))*i
#         folium.Circle(loc,color="red", popup='스타벅스', tooltip="",radius=10).add_to(my_map)
    return score
def foodScore(u,dif1,dif2,i):        # 위 함수와 같은 함수(지하철역)
    os.chdir(r"..\data1")
    wb = openpyxl.load_workbook('fastfood.xlsx')
    sh = wb['fastfood']
    score = 0
    for li in range(2,65):
        loc = [float(sh.cell(li,3).value) ,float(sh.cell(li,4).value)]
        if (a[1]-dif2<loc[1]<a[1]+dif2) &(a[0]-dif1<loc[0]<a[0]+dif1):
            score+=(0.5-get_dis(u,loc))*i
#         folium.Circle(loc,color="black", popup='스타벅스', tooltip="",radius=10).add_to(my_map)
    return score
def get_dis(loc1,loc2):        # 거리 구하는 함수
    a= loc1[0]-loc2[0]         #  위도 차이
    b = loc1[1]-loc2[1]        # 거리 구하는 함수
    radi = 6375                # 지구의 반지름
    pi =3.141592               # 원주율
    lat_dis = cos(loc1[0])*radi*a/180*pi    #가로방향 거리
    lng_dis = cos(loc1[1])*radi*b/180*pi    #세로방향 거리
    return sqrt(lat_dis**2+lng_dis**2)     #대각선방향 거리
def get_loc(addr):
    gmaps = googlemaps.Client(key="GOOGLE API KEY")  #구글맵 API Key입력       
    geocode_result = gmaps.geocode(addr)     #구글맵 geocode함수를 통해 주소로 정보 수집
    if not geocode_result :                 
        return
    n_lat = geocode_result[0]['geometry']['location']['lat']   #위도 출력
    n_lng = geocode_result[0]['geometry']['location']['lng']   #경도 출력
    loc = {'lat':n_lat, 'lng':n_lng} #딕셔너리로 반환
    return loc

In [19]:
my_map = folium.Map(location=[37.5007,127.04706],zoom_start=12.5)
score_list = []
folium.Choropleth(geo_data=geo_seoul,fill_opacity=0.2, fill_color="gray").add_to(my_map)

for a in (arr):
    score =0
    score +=conviScore(a,dif1,dif2,index[0])    #편의점 슬세권점수 
    score +=starBScore(a,dif1,dif2,index[1])    #스타벅스 슬세권점수 
    score +=subwayScore(a,dif1,dif2,index[2])   #지하철역 슬세권점수 
    score +=foodScore(a,dif1,dif2,index[3])     #패스트푸드 슬세권점수 
    score*=20                                   #원의 크기를 위해 값을 전체적으로 20배
    score_list.append(score)                    #score 배열에 추가'
my_score = score_list.pop()
sorted_score = sorted(score_list,reverse=True)  #score 배열을 내림차순으로 정리
for a , s in zip(arr,score_list):                 
    if s in sorted_score[:3]:                    #score 배열의 3위안에 드는 값들은 Marker로 표시
        folium.Marker(a, color = "red",popup=s, tooltip=s,radius=score,fill_color = "blue", fill = True).add_to(my_map)
    else :                                      #나머지는 Circle로 표시
        folium.Circle(a, popup=s, tooltip=s,radius=s,fill_opacity=0.7, fill = True).add_to(my_map)

In [20]:
my_map